In [11]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import StratifiedKFold, KFold,StratifiedShuffleSplit, cross_val_score, train_test_split
from sklearn.metrics import log_loss
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
import warnings
from scipy import stats
warnings.filterwarnings('ignore')

In [12]:
train = pd.read_csv('MPP_Dataset/Train.csv')
test = pd.read_csv('MPP_Dataset/Test.csv')

In [13]:
train.head()

train.popularity = train.popularity.replace(5,2) 

In [14]:
train.popularity.value_counts()

4    15231
3     2166
2      472
1      323
0       16
Name: popularity, dtype: int64

In [15]:
def onehotencoder(train,test,cols=[]):
    
    from sklearn.preprocessing import OneHotEncoder
    #object_cols = catVar1(train) #catVar1 gives desired categorical column and not all object columns
    object_cols=cols
    print(object_cols)
    
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train[object_cols]))
    OH_cols_test = pd.DataFrame(OH_encoder.transform(test[object_cols]))

    # One-hot encoding removed index; put it back
    OH_cols_train.index = train.index
    OH_cols_test.index = test.index

    ##hack for restoring columns names just like get dummies
    column_name = OH_encoder.get_feature_names(object_cols)
    OH_cols_train.columns = column_name
    OH_cols_test.columns = column_name
    

    # Remove desired categorical columns (will replace with one-hot encoding)
    num_train = train.drop(object_cols, axis=1)
    num_test = test.drop(object_cols, axis=1)

    # Add one-hot encoded columns to numerical/remaining features
    OH_train = pd.concat([num_train, OH_cols_train], axis=1)
    OH_test = pd.concat([num_test, OH_cols_test], axis=1)
    

    
    print(OH_train.shape,OH_test.shape)
    
    
    return OH_train,OH_test

def targetencoding(train,test,y_train,drop=False,cols=[]):
    import category_encoders as ce
    # Create the encoder itself
    cat_features = cols
    print(f'targest emcoding for features {cat_features}')
    target_enc = ce.TargetEncoder(cols=cat_features)

    

    # Fit the encoder using the categorical features and target
    target_enc.fit(train[cat_features], y_train)
    

    # Transform the features, rename the columns with _target suffix, and join to dataframe
    traintrgtenc = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
    testtrgtenc = test.join(target_enc.transform(test[cat_features]).add_suffix('_target'))

    if drop :
        traintrgtenc = traintrgtenc.drop(cat_features, axis =1)
        testtrgtenc = testtrgtenc.drop(cat_features, axis =1)

    
    print(traintrgtenc.shape,testtrgtenc.shape)
    
    return traintrgtenc,testtrgtenc

In [16]:
trainenc,testenc = targetencoding(train,test,train.popularity,drop=True,cols =['Category_1'])
trainencohe,testencohe = onehotencoder(trainenc,testenc,cols =['Category_2'])

targest emcoding for features ['Category_1']
(18208, 12) (12140, 11)
['Category_2']
(18208, 13) (12140, 12)


In [17]:
X = trainenc.drop('popularity',axis=1)
Y = trainenc['popularity']

In [18]:
# model = ExtraTreesClassifier(warm_start=True,oob_score=False,bootstrap=False ,random_state=786)

In [19]:
# param_rand = {
#   'ccp_alpha':  np.linspace(0.1, 1, num=10),
#  'criterion': ['gini','entropy'],
#  'max_depth':  range(2,25,1),
#  'max_features': ['auto','sqrt','log2'],
#  'max_leaf_nodes': range(1,100,5),
#  'min_samples_leaf': range(1,50,1),
#  'min_samples_split': range(1,20,1),
#  'n_estimators': range(450,700,25),
# }



In [20]:
# gsc = RandomizedSearchCV(
#     estimator=model,
#     param_distributions=param_rand,
#     scoring='neg_log_loss',
#     cv=5,
#     n_jobs=-1,
#     verbose =10)

# grid_result = gsc.fit(X, Y)

# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


In [21]:
# Best: -0.574999 using {'n_estimators': 475, 'min_samples_split': 15, 'min_samples_leaf': 17, 'max_leaf_nodes': 31, 'max_features': 'log2', 'max_depth': 11, 'criterion': 'gini', 'ccp_alpha': 0.5}

In [22]:
# param_rand = {
#   'ccp_alpha':  np.linspace(0.1, 1, num=10),
#  'criterion': ['gini','entropy'],
#  'max_depth':  range(2,25,1),
#  'max_features': ['auto','sqrt','log2'],
#  'max_leaf_nodes': range(1,100,5),
#  'min_samples_leaf': range(1,50,1),
#  'min_samples_split': range(1,20,1),
#  'n_estimators': range(450,700,25),
# }



In [23]:
# gsc = GridSearchCV(
#     estimator=model,
#     param_grid=param_rand,
#     scoring='neg_log_loss',
#     n_jobs=-1,
#     cv=5,
#  verbose =10)

# grid_result = gsc.fit(X, Y)

# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


In [ ]:


import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm

import optuna


# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    iris = sklearn.datasets.load_iris()
    x, y = iris.data, iris.target

    classifier_name = trial.suggest_categorical("classifier", ["SVC", "RandomForest"])
    if classifier_name == "SVC":
        svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
        classifier_obj = sklearn.svm.SVC(C=svc_c, gamma="auto")
    else:
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        classifier_obj = sklearn.ensemble.RandomForestClassifier(
            max_depth=rf_max_depth, n_estimators=10
        )

    score = sklearn.model_selection.cross_val_score(classifier_obj, x, y, n_jobs=-1, cv=3)
    accuracy = score.mean()
    return accuracy



study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_trial)

In [ ]:
https://towardsdatascience.com/exploring-optuna-a-hyper-parameter-framework-using-logistic-regression-84bd622cd3a5

In [10]:
scaler = StandardScaler()
X_ = scaler.fit_transform(X)
X = pd.DataFrame(X_,columns=X.columns)

folds = StratifiedKFold(n_splits=5,shuffle=False ,random_state=42)

oof_preds = []
oof_scores = []

for i , (train_idx,test_idx) in enumerate(folds.split(X,Y)):
  train_set = (X.iloc[train_idx],Y.iloc[train_idx])
  test_set = (X.iloc[test_idx],Y.iloc[test_idx])

  model = ExtraTreesClassifier(n_estimators=500,criterion= 'entropy',warm_start=True,
                                oob_score=False,bootstrap=False,max_features='auto',
                                random_state=2)

  model.fit(*train_set)

  preds = model.predict_proba(test_set[0])
  score = log_loss(test_set[1],preds)
  
  oof_pred_fold = model.predict_proba((test)) #scaler.transform
  oof_preds.append(oof_pred_fold)

  print(score)
  oof_scores.append(score)

print("Mean OOF Score :",np.mean(oof_scores))
final_preds = np.mean(np.array(oof_preds),axis=0)

0.37208834844703553
0.37869883476050376
0.34930671009905556
0.384452551047067
0.35798698250871047
Mean OOF Score : 0.36850668537247444


In [8]:
submission = pd.DataFrame(final_preds)
#submission.to_csv("submission4.csv",index=None)
submission

,0,1,2,3,4
0,0.0,0.058333,0.280333,0.545000,0.116333
1,0.0,0.046667,0.256667,0.596000,0.100667
2,0.0,0.044333,0.275333,0.593000,0.087333
3,0.0,0.051333,0.260000,0.585667,0.103000
4,0.0,0.052667,0.248333,0.608667,0.090333
...,...,...,...,...,...
12135,0.0,0.030667,0.320667,0.539000,0.109667
12136,0.0,0.046333,0.262667,0.614000,0.077000
12137,0.0,0.056333,0.239333,0.608333,0.096000
12138,0.0,0.045333,0.246333,0.621667,0.086667


In [122]:
submission[0].sum()

9.6844